# Reinforcement Learning mit Python - Running Kings Schach KI
## Aufgabenstellung

Erstellen  Sie  Modelle,  welche  das  Spiel  Racing  Kings erlernen.  Am  Ende  sollte  ein  Spiel  gegen  den erstellten  Algorithmus  möglich  sein.  Testen  Sie  als  Gegner  einenAlgorithmus,  welcher  Zufallszüge ausführt,  und  bewerten  Sie  ihren  Algorithmus. Nutzen  Sie  die  chess-Bibliothek  (https://python-chess.readthedocs.io/en/latest/)

Bonus: Vergleichen Sie einen modellfreien und einen modellbasierten Ansatz.

Diese Anleitung wurde verwendet um zügig mit der verwendung der chess bibliothek starten zu können.
https://jupyter.brynmawr.edu/services/public/dblank/CS371%20Cognitive%20Science/2016-Fall/Programming%20a%20Chess%20Player.ipynb

# Install Dependencies

In [2]:
!pip install numpy
!pip install tensorflow==2.3.0
!pip install keras
!pip install keras-rl2
!pip install chess
!pip install pydot
!pip install gym
!pip install stable_baselines3

You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.


# Create chess environment

In [3]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

In [4]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

In [37]:
import chess.variant
import random
import time
from IPython.display import display, HTML, clear_output
import numpy as np
import gym
from gym.spaces import Discrete, Box, Dict

N_DISCRETE_ACTIONS = 4096

# from Learning_Chess pdf
class RacingKingsEnvironment(gym.Env):
    def __init__(self):
        super(RacingKingsEnvironment, self).__init__()
        self.board = chess.variant.RacingKingsBoard()
        self.reward = 0
        self.action_space = Discrete(N_DISCRETE_ACTIONS)
        self.observation_space = Box(low=0, high=1, shape=(14, 8, 8), dtype=np.uint8)
    def board_square_to_index(self, name):
        return (int(name[1])-1) * 8 + (ord(name[0])-97) 
    def action_index_to_uci(self, index):
        index_from = index//64
        index_to = index%64
        name = chr(index_from%8 + 97) + str(index_from//8 +1) + chr(index_to%8 + 97) + str(index_to//8 + 1)
        return name
    def action_uci_to_index(self, uci):
        index_from = (int(uci[1])-1) * 8 + (ord(uci[0])-97) 
        index_to = (int(uci[3])-1) * 8 + (ord(uci[2])-97) 
        return index_from*64 + index_to
    @property
    def actions(self):
        moves = list(self.board.legal_moves)
        moves_string = []
        for move in moves:
            moves_string.append(move.uci())
        boardActions = np.zeros(N_DISCRETE_ACTIONS, dtype=np.int8)
        for move in moves_string:
            boardActions[self.board_square_to_index(move[0:2])*64 + self.board_square_to_index(move[2:4])] = 1
        return boardActions
    @property
    def states(self):
        boardState = np.zeros((14, 8, 8), dtype=np.int8)
        for piece in chess.PIECE_TYPES:
            for square in self.board.pieces(piece, chess.WHITE):
                idNum = square//8
                idAlph = square%8
                boardState[piece - 1][7 - idNum][idAlph] = 1
            for square in self.board.pieces(piece, chess.BLACK):
                idNum = square//8
                idAlph = square%8
                boardState[piece + 5][7 - idNum][idAlph] = 1        
        
            aux = self.board.turn
            self.board.turn = chess.WHITE
            for move in list(self.board.legal_moves):
                square = self.board_square_to_index(move.uci())
                idNum = square//8
                idAlph = square%8
                boardState[12][7 - idNum][idAlph] = 1
            self.board.turn = chess.BLACK
            for move in list(self.board.legal_moves):
                square = self.board_square_to_index(move.uci())
                idNum = square//8
                idAlph = square%8
                boardState[13][7 - idNum][idAlph] = 1
            self.board.turn = aux
        return boardState
    
    def step(self, action):
        done = False
        step_reward = 0
        info = {}
        
        # check if it is not your turn -> then make a random move
        # when uncommenting this code the ai will play moves for both players
        if self.board.turn == chess.WHITE:
            try:
                self.board.push(random.choice(list(self.board.legal_moves)))
                step_reward += 0
                info = {"msg":"White Did a valid move"}
            except:
                info = {"msg":"Passed an already finished board"} 
                done = True
           

        if not self.board.is_game_over(claim_draw=True):
            if action is not None:
                try:
                    self.board.push_uci(self.action_index_to_uci(action))
                    step_reward += 0.1
                    info = {"msg":"Did a valid move"}
                except:
                    step_reward -= 1
                    info = {"msg":"Action is not a valid move"} 
                    done = True
                if self.board.is_variant_end():
                    if who(not self.board.turn) == "Black":
                        step_reward += 100
                        info = {"msg":"AI won the game!"} 
                        done = True
                    else:
                        step_reward -= 100
                        info = {"msg":"Opponent won the game!"} 
                        done = True
                    info = {"msg":"racing kings: " + who(not self.board.turn) + " wins!"}
        else:
            step_reward -=10
            done = True
            info = {"msg":"game over"}

        self.reward += step_reward
        
        return self.states, step_reward, done, info
    def reset(self):
        # reset the board
        self.board.reset()
        # play random amount of actions
        for i in range((random.randint(0, 30)*2)):
            try:
                move = random.choice(list(self.board.legal_moves))
                self.board.push(move)
            except:
                self.board.reset()
                
        self.reward = 0.0
        return self.step(None)[0]  # reward, done, info can't be included
    def render(self, mode="human", pause=0.2):
        name = who(self.board.turn)
        use_svg = (mode == "human")
        board_stop = display_board(self.board, use_svg)
        html = "<b>Move %s %s:</b><br/>%s" % (
                    len(self.board.move_stack), name, board_stop)
        if mode is not None:
                if mode == "human":
                    clear_output(wait=True)
                display(HTML(html))
                if mode == "human":
                    time.sleep(pause)
    def close (self):
        print("closing")

# Test Chess environment randomly

In [38]:
episodes = 10
env = RacingKingsEnvironment()
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render(mode=None)
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} Info:{}'.format(episode, score, info))
env.close()

Episode:1 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:2 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:3 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:4 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:5 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:6 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:7 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:8 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:9 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:10 Score:-1 Info:{'msg': 'Action is not a valid move'}
closing


# Training Model with stable baselines

In [39]:
from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy 
from stable_baselines3.common.callbacks import EvalCallback 
import os

In [40]:
log_path = os.path.join('Training', 'Logs')

In [69]:
env = RacingKingsEnvironment()
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [70]:
best_save_path = os.path.join('Training', 'SavedModels', 'Best_Racing_Kings')
eval_callback = EvalCallback(env, eval_freq=2000, best_model_save_path = best_save_path, verbose=1)

In [73]:
model.learn(total_timesteps=1000000, callback=eval_callback)

Logging to Training/Logs/PPO_31
Eval num_timesteps=2000, episode_reward=-0.70 +/- 0.37
Episode length: 4.00 +/- 3.69
---------------------------------
| eval/              |          |
|    mean_ep_length  | 4        |
|    mean_reward     | -0.7     |
| time/              |          |
|    total timesteps | 2000     |
---------------------------------
New best mean reward!
-----------------------------
| time/              |      |
|    fps             | 50   |
|    iterations      | 1    |
|    time_elapsed    | 40   |
|    total_timesteps | 2048 |
-----------------------------
Eval num_timesteps=4000, episode_reward=0.12 +/- 1.37
Episode length: 12.20 +/- 13.67
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 12.2       |
|    mean_reward          | 0.12       |
| time/                   |            |
|    total timesteps      | 4000       |
| train/                  |            |
|    approx_kl            | 0.08392431 |

------------------------------
| time/              |       |
|    fps             | 45    |
|    iterations      | 9     |
|    time_elapsed    | 402   |
|    total_timesteps | 18432 |
------------------------------
Eval num_timesteps=20000, episode_reward=-0.88 +/- 0.12
Episode length: 2.20 +/- 1.17
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 2.2        |
|    mean_reward          | -0.88      |
| time/                   |            |
|    total timesteps      | 20000      |
| train/                  |            |
|    approx_kl            | 0.08856416 |
|    clip_fraction        | 0.249      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.656     |
|    explained_variance   | 0.169      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0235    |
|    n_updates            | 4980       |
|    policy_gradient_loss | -0.0388    |
|    value_loss           | 0.106      |


------------------------------
| time/              |       |
|    fps             | 47    |
|    iterations      | 17    |
|    time_elapsed    | 735   |
|    total_timesteps | 34816 |
------------------------------
Eval num_timesteps=36000, episode_reward=-0.66 +/- 0.28
Episode length: 4.40 +/- 2.80
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 4.4        |
|    mean_reward          | -0.66      |
| time/                   |            |
|    total timesteps      | 36000      |
| train/                  |            |
|    approx_kl            | 0.06287682 |
|    clip_fraction        | 0.219      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.655     |
|    explained_variance   | 0.126      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0137    |
|    n_updates            | 5060       |
|    policy_gradient_loss | -0.0313    |
|    value_loss           | 0.204      |


------------------------------
| time/              |       |
|    fps             | 48    |
|    iterations      | 25    |
|    time_elapsed    | 1062  |
|    total_timesteps | 51200 |
------------------------------
Eval num_timesteps=52000, episode_reward=0.94 +/- 2.14
Episode length: 20.40 +/- 21.43
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 20.4        |
|    mean_reward          | 0.94        |
| time/                   |             |
|    total timesteps      | 52000       |
| train/                  |             |
|    approx_kl            | 0.053522736 |
|    clip_fraction        | 0.18        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.664      |
|    explained_variance   | 0.0141      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.113       |
|    n_updates            | 5140        |
|    policy_gradient_loss | -0.023      |
|    value_loss         

------------------------------
| time/              |       |
|    fps             | 48    |
|    iterations      | 33    |
|    time_elapsed    | 1397  |
|    total_timesteps | 67584 |
------------------------------
Eval num_timesteps=68000, episode_reward=-0.60 +/- 0.24
Episode length: 5.00 +/- 2.45
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 5          |
|    mean_reward          | -0.6       |
| time/                   |            |
|    total timesteps      | 68000      |
| train/                  |            |
|    approx_kl            | 0.06875014 |
|    clip_fraction        | 0.257      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.9       |
|    explained_variance   | 0.0251     |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0113     |
|    n_updates            | 5220       |
|    policy_gradient_loss | -0.0315    |
|    value_loss           | 0.316      |


...

-------------------------------
| time/              |        |
|    fps             | 46     |
|    iterations      | 457    |
|    time_elapsed    | 20058  |
|    total_timesteps | 935936 |
-------------------------------
Eval num_timesteps=936000, episode_reward=-1.00 +/- 0.00
Episode length: 1.00 +/- 0.00
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 1          |
|    mean_reward          | -1         |
| time/                   |            |
|    total timesteps      | 936000     |
| train/                  |            |
|    approx_kl            | 0.08330257 |
|    clip_fraction        | 0.284      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.758     |
|    explained_variance   | 0.299      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0232    |
|    n_updates            | 9460       |
|    policy_gradient_loss | -0.043     |
|    value_loss           | 0.208

-------------------------------
| time/              |        |
|    fps             | 46     |
|    iterations      | 465    |
|    time_elapsed    | 20422  |
|    total_timesteps | 952320 |
-------------------------------
Eval num_timesteps=954000, episode_reward=-0.92 +/- 0.16
Episode length: 1.80 +/- 1.60
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 1.8        |
|    mean_reward          | -0.92      |
| time/                   |            |
|    total timesteps      | 954000     |
| train/                  |            |
|    approx_kl            | 0.09444274 |
|    clip_fraction        | 0.312      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.853     |
|    explained_variance   | 0.353      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0105    |
|    n_updates            | 9540       |
|    policy_gradient_loss | -0.0488    |
|    value_loss           | 0.192

-------------------------------
| time/              |        |
|    fps             | 46     |
|    iterations      | 473    |
|    time_elapsed    | 20779  |
|    total_timesteps | 968704 |
-------------------------------
Eval num_timesteps=970000, episode_reward=-0.42 +/- 0.82
Episode length: 6.80 +/- 8.18
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 6.8         |
|    mean_reward          | -0.42       |
| time/                   |             |
|    total timesteps      | 970000      |
| train/                  |             |
|    approx_kl            | 0.068945035 |
|    clip_fraction        | 0.22        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.737      |
|    explained_variance   | -0.0124     |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0191     |
|    n_updates            | 9620        |
|    policy_gradient_loss | -0.0259     |
|    value_loss  

-------------------------------
| time/              |        |
|    fps             | 46     |
|    iterations      | 481    |
|    time_elapsed    | 21136  |
|    total_timesteps | 985088 |
-------------------------------
Eval num_timesteps=986000, episode_reward=-0.94 +/- 0.12
Episode length: 1.60 +/- 1.20
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 1.6        |
|    mean_reward          | -0.94      |
| time/                   |            |
|    total timesteps      | 986000     |
| train/                  |            |
|    approx_kl            | 0.11377078 |
|    clip_fraction        | 0.301      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.765     |
|    explained_variance   | 0.129      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0186    |
|    n_updates            | 9700       |
|    policy_gradient_loss | -0.0506    |
|    value_loss           | 0.21 

--------------------------------
| time/              |         |
|    fps             | 46      |
|    iterations      | 489     |
|    time_elapsed    | 21512   |
|    total_timesteps | 1001472 |
--------------------------------


Saving the model

In [44]:
Model_Path = os.path.join('Training', 'SavedModels', 'PPO_Racing_Kings_1m_win+step_reward_random_start_after_reset')

In [45]:
model.save(Model_Path)

In [71]:
del model

In [72]:
model = PPO.load(Model_Path, env=env)

# Evaluate the Model

# Storing the model weights

In [ ]:
dqn.save_weights('saved_weights/100k_dqn_weights.h5f')

In [ ]:
def random_player(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()
for i in range(10):
    print(random_player(board))

In [ ]:
play_game(random_player, random_player)

In [ ]:
counts = {None: 0, True: 0, False: 0}
for i in range(10):
    result, msg, board = play_game(random_player, random_player, visual=None)
    counts[result] += 1
    print(counts)
counts

# Testing the Model

In [62]:
episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render(mode=None)
        action = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} Info:{}'.format(episode, score, info))

Episode:1 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:2 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:3 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:4 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:5 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:6 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:7 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:8 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:9 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:10 Score:-1 Info:{'msg': 'Action is not a valid move'}


In [67]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(-0.5499999932944775, 0.854692940049581)